<a href="https://colab.research.google.com/github/ELehmann91/NLP-Contract-Analysis/blob/master/Coop_ktrain_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%%capture
!pip install ktrain
!pip3 install git+https://github.com/amaiya/eli5@tfkeras_0_10_1

In [0]:
import ktrain
from ktrain import text

In [0]:
from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import pandas as pd
path = '/content/gdrive/Shared drives/FS-AI Base Data/'
df = pd.read_csv(path+'df_bert.csv')

DATA_PATH = '/content/gdrive/Shared drives/FS-AI Base Data/df_bert.csv'
NUM_WORDS = 50000
MAXLEN = 250

In [0]:
labels = [a for a in df.columns if a[:3]=='012']
labels

['012_business_day_convention_',
 '012_business_day_convention_Following adjusted',
 '012_business_day_convention_Following unadjusted',
 '012_business_day_convention_Modified following adjusted',
 '012_business_day_convention_Unadjusted']

In [0]:
(x_train, y_train), (x_test, y_test), preproc = text.texts_from_csv(DATA_PATH, 
                                                                   'pdf_text', # name of column containing review text
                                                                   label_columns=labels, #['Bond','Schuldschein','other Loan','Covered Bond'],
                                                                   max_features=NUM_WORDS, maxlen=MAXLEN,
                                                                   lang='ze',
                                                                   val_filepath=None,
                                                                   preprocess_mode='bert')

detected encoding: utf-8 (if wrong, set manually)
preprocessing train...
language: ze


preprocessing test...
language: ze


In [0]:
model = text.text_classifier('bert', (x_train, y_train) , preproc=preproc)
learner = ktrain.get_learner(model, 
                             train_data=(x_train, y_train), 
                             val_data=(x_test, y_test), 
                             batch_size=1)

Is Multi-Label? False
maxlen is 250
done.


In [0]:
learner.fit_onecycle(2e-5, 3)



begin training using onecycle policy with max lr of 2e-05...
Train on 134 samples, validate on 15 samples
Epoch 1/3
134/134 [==============================] - 521s 4s/sample - loss: 0.5833 - acc: 0.7015 - val_loss: 0.5687 - val_acc: 0.7333
Epoch 2/3
134/134 [==============================] - 507s 4s/sample - loss: 0.4673 - acc: 0.7687 - val_loss: 0.4261 - val_acc: 0.7333
Epoch 3/3
134/134 [==============================] - 506s 4s/sample - loss: 0.3812 - acc: 0.8284 - val_loss: 0.3574 - val_acc: 0.8000


In [0]:
learner.validate(val_data=(x_train, y_train))

              precision    recall  f1-score   support

           0       0.86      0.98      0.92        95
           1       0.92      0.62      0.74        39

    accuracy                           0.87       134
   macro avg       0.89      0.80      0.83       134
weighted avg       0.88      0.87      0.86       134



array([[93,  2],
       [15, 24]])

In [0]:
learner.validate(val_data=(x_test, y_test))

              precision    recall  f1-score   support

           0       0.67      0.67      0.67         6
           1       0.80      0.80      0.80         5
           2       0.00      0.00      0.00         0
           3       1.00      0.67      0.80         3

    accuracy                           0.71        14
   macro avg       0.62      0.53      0.57        14
weighted avg       0.79      0.71      0.74        14



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


array([[4, 1, 1, 0],
       [1, 4, 0, 0],
       [0, 0, 0, 0],
       [1, 0, 0, 2]])

In [0]:
predictor = ktrain.get_predictor(learner.model, preproc)
predictor.get_classes()

['001_type_Fixed', '001_type_Floating']

In [0]:
data=['2% jedes Jahr','Euribor CMS','Libor du blöde kuh']
predictor.predict(data)

['001_type_Fixed', '001_type_Fixed', '001_type_Fixed']

In [0]:
predictor.save('/tmp/my_moviereview_predictor')
predictor = ktrain.load_predictor('/tmp/my_moviereview_predictor')

In [0]:
learner.view_top_losses(n=1, preproc=preproc)

----------
id:7 | loss:1.21 | true:001_type_Floating | pred:001_type_Fixed)

[CLS] final terms dated 16 september 2019 ##nede ##rland ##se waters ##chap ##s ##bank n . v . ( incorporated under the l ##ø , vs of the net ##her ##lands with limited li ##ability and having its corporate seat in the ##ha ##gue ) legal entity ide ##nti ##fier : j ##lp ##5 ##fs ##ph ##9 ##w ##ps [ ry ##3 ##nim ##24 ##iss ##ue of us $ 100 , 000 , 000 re ##g sl ##l ##44 ^ floating rate notes due 10 november 2020 ( the ' notes ' ) ( to be con ##solidated and form a single series with the outstanding issues of us $ 500 , 000 , 000 re ##gs / l ##44 ##af ##loa ##ting rate notes due 10 november 2020 issued on 10 may 2019 , us $ 50 , 000 , 000 re ##gs / 1444 ##f ##loa ##ting rate notes due 10 november 2020 issued on 11 july 2019 , us $ 250 , 000 , 000 re ##g 5 / 144 . 4 . floating rate notes due 10 november 2020 issued on 16 july 2019 , us $ 200 , 000 , 000 re ##g 5 / 1444 ##f ##loa ##ting rate notes due 10 november 

In [0]:
doc = df['pdf_text'][1][1000:1550]
doc

'ain  Rating A (Standard and Poor`s), Aa2 (Moody`s)  Issue Size EUR 100,000,000 (1,000, 000 certificates á EUR 100,--)   Issue Price 100%  Minimum Trading Size 100 EUR (= 1 certificate)  Underlying  10 years EURO - Swap minus 2 years EURO Swap according to ISDAFIX2   11 a.m. Frankfurt  Issue Date 23 July 2008  Issue Payment Date 25 July 2008   Maturity 21 September 2012, subject to an early redemption   Observation dates 1) 21 September 2009  2) 21 September 2010   3) 21 September 2011   4) 21 September 2012  Redemption Payment date  5 bank busi'

In [0]:

predictor.explain(doc)

KeyboardInterrupt: ignored

  Cloning https://github.com/amaiya/eli5 (to revision tfkeras_0_10_1) to /tmp/pip-req-build-ohr3rjjc
  Running command git clone -q https://github.com/amaiya/eli5 /tmp/pip-req-build-ohr3rjjc
  Running command git checkout -b tfkeras_0_10_1 --track origin/tfkeras_0_10_1
  Switched to a new branch 'tfkeras_0_10_1'
  Branch 'tfkeras_0_10_1' set up to track remote branch 'tfkeras_0_10_1' from 'origin'.
  Created wheel for eli5: filename=eli5-0.10.1-py2.py3-none-any.whl size=106682 sha256=83c9ee380fd10d77efe41d970320ef3598cfb3304cc370d2056585858ed8989f
  Stored in directory: /tmp/pip-ephem-wheel-cache-w2lva285/wheels/51/59/0a/0f48442b8d209583a4453580938d7ba2270aca40edacee6d45
Successfully built eli5
